In [2]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import random
import sklearn
import collections
from sklearn.model_selection import train_test_split
import json
import pylab
import tensorflow as tf
import tensorflow.math as tfmath
import tensorflow.keras as keras
from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import sklearn.metrics as sk

2024-06-04 21:19:00.126657: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-04 21:19:00.320366: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class Sampling(keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


def make_encoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(input_dim))
    x=layers.BatchNormalization()(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    x=layers.Dense(h_dim_2, activation='relu')(x)
    z_mean=layers.Dense(latent_dim, name='z_mean')(x)
    z_logvar=layers.Dense(latent_dim, name='z_log_var',kernel_initializer=keras.initializers.Zeros())(x)
    z=Sampling()([z_mean,z_logvar])
    encoder=keras.Model(inputs,[z_mean,z_logvar,z],name='encoder')
    return encoder




def make_decoder(input_dim,h_dim_1,h_dim_2,latent_dim):
    inputs=keras.Input(shape=(latent_dim))
    x=layers.Dense(h_dim_2, activation='relu')(inputs)
    x=layers.Dense(h_dim_1, activation='relu')(x)
    z=layers.Dense(input_dim)(x)
    decoder=keras.Model(inputs,z,name='decoder')
    return decoder

class VAE_Model(keras.Model):
    def __init__(self,encoder,decoder,**kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.beta=1

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]
    
    def set_beta(self,beta):
        self.beta=beta
    
    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            #making a masked loss function
            mask = K.cast(K.not_equal(data, 0), K.floatx())
            
            reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
            reconstruction_loss *=(1-self.beta)

            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *=self.beta


            total_loss = reconstruction_loss + kl_loss
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reco_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def test_step(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        
        reconstruction = self.decoder(z)
        mask = K.cast(K.not_equal(data, 0), K.floatx())
        reconstruction_loss = tf.reduce_mean(tf.reduce_sum(keras.losses.mse(mask*data, mask*reconstruction)))
        
        reconstruction_loss*=(1-self.beta)

        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        
        #KL loss changed abck to sum as in paper
        kl_loss = tf.reduce_sum(kl_loss)
        
        kl_loss *=self.beta
        
        total_loss = reconstruction_loss + kl_loss
        
        return {
            "loss": total_loss,
            "reco_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, data):
        z_mean,z_log_var,x = self.encoder(data)
        reconstruction = self.decoder(x)
        return {
            "z_mean": z_mean,
            "z_log_var": z_log_var,
            "reconstruction": reconstruction
        }

        
 

In [4]:
with h5py.File('/eos/home-m/mmcohen/ntuples/AOD_EB_ntuples_05-23-2024.h5','r') as hf:
    HLT_jets = hf['HLT_jets'][:]
    L1_jFexSR_jets = hf['L1_jFexSR_jets'][:,0:6,:]
    print(np.array(L1_jFexSR_jets).shape)
    L1_jFexLR_jets = hf['L1_jFexLR_jets'][:,0:6,:]
    print(np.array(L1_jFexLR_jets).shape)
    HLT_electrons = hf['HLT_electrons'][:]
    LRT_electrons = hf['LRT_electrons'][:]
    L1_egammas = hf['L1_egammas'][:]
    HLT_muons = hf['HLT_muons'][:]
    LRT_muons = hf['LRT_muons'][:]
    L1_muons = hf['L1_muons'][:]
    L1_eFex_taus = hf['L1_eFex_taus'][:]
    L1_jFex_taus = hf['L1_jFex_taus'][:]
    HLT_photons = hf['HLT_photons'][:]
    print(np.array(hf['HLT_MET']).shape)
    print(np.array(hf['L1_MET']).shape)
    print(np.array(hf['LRT_muons']).shape)
    HLT_MET = hf['HLT_MET'][:].reshape(-1, 1, 3)  # Broadcasting MET
    L1_MET = hf['L1_MET'][:].reshape(-1, 1, 3)
    #print(L1_MET.shape)
    #print(L1_MET[0,:,:])
    L1_MET_fixed=np.zeros((2059879, 1, 3))
    for i,event in enumerate(L1_MET):
        L1_MET_fixed[i,0,0:3]=event[0,0:3]
    #print(L1_MET_fixed.shape)
    #print(L1_MET_fixed[0,:,:])
    L1_MET=L1_MET_fixed
    pass_L1_unprescaled = hf["pass_L1_unprescaled"][:]
    pass_HLT_unprescaled = hf["pass_HLT_unprescaled"][:]
    EB_weights = hf["EB_weights"][:]
    e_num=hf['event_number'][:]
    r_num=hf['run_number'][:]
    print(np.array(hf['run_number']))
    print(list(hf['run_number']).index(436169))

Topo_2A=[L1_muons,L1_eFex_taus,L1_jFex_taus,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET,L1_egammas]
Topo_2A_labels=['L1_muons','L1_eFex_taus','L1_jFex_taus','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET','L1_egammas']
#Topo_2B=[L1_egammas,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET,L1_jFex_taus]
#Topo_2B_labels=['L1_egammas','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET','L1_jFex_taus']
#Topo_3A=[L1_egammas,L1_eFex_taus,L1_jFex_taus,L1_jFexLR_jets,L1_jFexSR_jets,L1_MET]
#Topo_3A_labels=['L1_egammas','L1_eFex_taus','L1_jFex_taus','L1_jFexLR_jets','L1_jFexSR_jets','L1_MET']
#Topo_3B=[L1_muons,L1_egamma,L1_eFex_taus]


Topo_2A_concat=Topo_2A[0]
for i in Topo_2A[1:]:
    Topo_2A_concat=np.append(Topo_2A_concat,i,axis=1)
Topo_2A_flat=np.reshape(Topo_2A_concat,(-1,111))

#Topo_2B_concat=Topo_2B[0]
#for i in Topo_2B[1:]:
#    Topo_2B_concat=np.append(Topo_2B_concat,i,axis=1)
#Topo_2B_flat=np.reshape(Topo_2B_concat,(-1,75))

#Topo_3A_concat=Topo_3A[0]
#for i in Topo_3A[1:]:
#    Topo_3A_concat=np.append(Topo_3A_concat,i,axis=1)
#Topo_3A_flat=np.reshape(Topo_3A_concat,(-1,93))

(2059879, 6, 3)
(2059879, 6, 3)
(2059879, 3)
(2059879, 3)
(2059879, 6, 3)
[440499 440499 440499 ... 436169 436169 436169]
1012567


In [5]:
nans=np.isnan(Topo_2A_flat)
a=np.where(nans==True)[0]
for i in a:
    Topo_2A_flat[i,92]=0

#1012567

Topo_2A_test_1=Topo_2A_flat[0:500000,:]
Topo_t_weights_1=EB_weights[0:500000]

Topo_2A_test_2=Topo_2A_flat[1012567:1012567+500000,:]
Topo_t_weights_2=EB_weights[1012567:1012567+500000]

 



In [23]:
#TOPO 2A Trainer 1

beta=0.85
T2A_enc=make_encoder(111,32,16,3)
T2A_dec=make_decoder(111,32,16,3)
T2A=VAE_Model(T2A_enc,T2A_dec)
T2A.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
T2A.compile(optimizer=opt)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = T2A.fit(x=Topo_2A_test_1, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
T2A.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_2A/Topo_2A_max_test_1/',save_format='tf')

Epoch 1/100
4500/4500 [==============================] - 14s 3ms/step - loss: 239.9540 - reco_loss: 211.4064 - kl_loss: 13.2360 - val_loss: 1354.1080 - val_reco_loss: 69.2223 - val_kl_loss: 1284.8857 - lr: 0.0010
Epoch 2/100
4500/4500 [==============================] - 11s 2ms/step - loss: 76.2209 - reco_loss: 72.0192 - kl_loss: 3.9135 - val_loss: 1043.0239 - val_reco_loss: 64.6252 - val_kl_loss: 978.3987 - lr: 0.0010
Epoch 3/100
4500/4500 [==============================] - 11s 2ms/step - loss: 69.6511 - reco_loss: 66.3425 - kl_loss: 3.2440 - val_loss: 994.8373 - val_reco_loss: 61.8729 - val_kl_loss: 932.9644 - lr: 0.0010
Epoch 4/100
4500/4500 [==============================] - 11s 2ms/step - loss: 69.2268 - reco_loss: 66.1467 - kl_loss: 3.0252 - val_loss: 882.2655 - val_reco_loss: 59.4485 - val_kl_loss: 822.8170 - lr: 0.0010
Epoch 5/100
4500/4500 [==============================] - 11s 2ms/step - loss: 66.8014 - reco_loss: 63.9296 - kl_loss: 2.8174 - val_loss: 864.9444 - val_reco_loss:

In [24]:
beta=0.85
T2A_enc=make_encoder(111,32,16,3)
T2A_dec=make_decoder(111,32,16,3)
T2A1=VAE_Model(T2A_enc,T2A_dec)
T2A1.set_beta(beta)
opt=keras.optimizers.Adam(learning_rate=0.001)
T2A.compile(optimizer=opt)

STOP_PATIENCE = 8
LR_PATIENCE = 4

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=LR_PATIENCE, verbose=1)

callbacks = [early_stopping, reduce_lr]

NUM_EPOCHS = 100
BATCH_SIZE = 100

history = T2A.fit(x=Topo_2A_test_2, validation_split=0.1,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks,shuffle=True,sample_weight=Topo_t_weights)
T2A.save_weights(filepath='/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_2A/Topo_2A_max_test_2/',save_format='tf')

Epoch 1/100
4500/4500 [==============================] - 14s 3ms/step - loss: 49.6970 - reco_loss: 47.0592 - kl_loss: 2.6159 - val_loss: 852.1293 - val_reco_loss: 56.0225 - val_kl_loss: 796.1068 - lr: 0.0010
Epoch 2/100
4500/4500 [==============================] - 11s 2ms/step - loss: 47.4732 - reco_loss: 44.9043 - kl_loss: 2.5541 - val_loss: 828.8904 - val_reco_loss: 51.5518 - val_kl_loss: 777.3386 - lr: 0.0010
Epoch 3/100
4500/4500 [==============================] - 11s 2ms/step - loss: 45.8958 - reco_loss: 43.4007 - kl_loss: 2.4983 - val_loss: 832.2730 - val_reco_loss: 55.7440 - val_kl_loss: 776.5290 - lr: 0.0010
Epoch 4/100
4500/4500 [==============================] - 11s 2ms/step - loss: 45.1766 - reco_loss: 42.6971 - kl_loss: 2.4902 - val_loss: 832.4368 - val_reco_loss: 57.3000 - val_kl_loss: 775.1369 - lr: 0.0010
Epoch 5/100
4500/4500 [==============================] - 11s 3ms/step - loss: 45.7975 - reco_loss: 43.3192 - kl_loss: 2.4689 - val_loss: 825.2184 - val_reco_loss: 52.41

In [6]:
def total_loss(signal,predict,z_mean,z_log_var,beta):
    re_loss=np.mean((signal-predict)**2)
    kl_loss=np.sum(-0.5 * (1 + z_log_var - (z_mean)**2 - np.exp(z_log_var)))
    tot_loss=re_loss+beta*kl_loss
    return tot_loss
def AD_score(s,p,z_m,z_log,beta):
    mask = (s!=0)
    
    _s=s*mask
    _p=p*mask
    
    p=mask*p
    return total_loss(_s,_p,z_m,z_log,beta)

def AD_score_MSE(s,p):
    mask = (s!=0)
    s1=s*mask
    p1=p*mask
    re_loss=np.mean((s1-p1)**2)
    return re_loss

def AD_score_KL(z_mean,z_log_var):
    kl_loss=np.mean(-0.5 * (1 + z_log_var - (z_mean)**2 - np.exp(z_log_var)))
    return kl_loss

def AD_score_Rz(z_mean,z_log_var):
    Rz_loss=np.mean((z_mean**2)/(np.exp(z_log_var)))
    return Rz_loss

def AD_score_CKL(z_mean,z_log_var):
    CKL=np.mean(z_mean**2)
    return CKL
    

class Model_Evaluator():
  def __init__(self,model_path,backround,br_weights,signal,signal_weights,input_dim,title='placeholder',save=False,labels=None):

    self.input_dim=input_dim
    vae_enc=make_encoder(input_dim,32,16,3)
    vae_dec=make_decoder(input_dim,32,16,3)
    #vae_enc=make_encoder2(57,16,3)
    #vae_dec=make_decoder2(57,16,3)
    self.model=VAE_Model(vae_enc,vae_dec)
    self.model.load_weights(model_path)
    self.encoder=self.model.get_layer('encoder')
    self.signal=signal
    self.backround=backround
    self.br_loss=[]
    self.signal_loss=[]
    self.backround_outputs=[]
    self.signal_outputs=[]
    self.title=title
    self.saveplots=save
    self.labels=labels
    self.latent_info=[]
    self.br_weights=br_weights
    self.signal_weights=signal_weights
  def calculate_loss(self,l_type):
    self.signal_loss=[]
    self.br_loss=[]
    br=self.backround
    
    if l_type=='CKL':
        br_latent=np.array(self.encoder.predict(br))
        self.latent_info+=[br_latent[0]]
        l=[]
        for i in range(0,br.shape[0]):
            loss=AD_score_CKL(br_latent[0][i],br_latent[1][i])
            l+=[loss]
        self.br_loss= l

        
        for i, batch in enumerate(self.signal):
            sg_latent=np.array(self.encoder.predict(batch))
            self.latent_info+=[sg_latent[0]]
            l=[]
        
            for i in range(0,batch.shape[0]):
                loss=AD_score_CKL(sg_latent[0][i],sg_latent[1][i])
                l+=[loss]
    
            sg_loss= l
        
            self.signal_loss+=[sg_loss]            
            
            

    
    
    if l_type=='KL':
        br_latent=np.array(self.encoder.predict(br))
        l=[]
        for i in range(0,br.shape[0]):
            loss=AD_score_KL(br_latent[0][i],br_latent[1][i])
            l+=[loss]
        self.br_loss= l

        
        for i, batch in enumerate(self.signal):
            sg_latent=np.array(self.encoder.predict(batch))
        
            l=[]
        
            for i in range(0,batch.shape[0]):
                loss=AD_score_KL(sg_latent[0][i],sg_latent[1][i])
                l+=[loss]
    
            sg_loss= l
        
            self.signal_loss+=[sg_loss]
    
    if l_type=='Rz':
        br_latent=np.array(self.encoder.predict(br))
        l=[]
        for i in range(0,br.shape[0]):
            loss=AD_score_Rz(br_latent[0][i],br_latent[1][i])
            l+=[loss]
        self.br_loss= l

        
        for i, batch in enumerate(self.signal):
            sg_latent=np.array(self.encoder.predict(batch))
        
            l=[]
        
            for i in range(0,batch.shape[0]):
                loss=AD_score_Rz(sg_latent[0][i],sg_latent[1][i])
                l+=[loss]
    
            sg_loss= l
        
            self.signal_loss+=[sg_loss]
    
    
    
    
    if l_type=='MSE':
        br_predict=np.array(self.model.predict(br)['reconstruction'])
        l=[]
        for i in range(0,br.shape[0]):
            loss=AD_score_MSE(br[i],br_predict[i])
            l+=[loss]
        self.br_loss= l
        
        for i, batch in enumerate(self.signal):
            sg_predict=np.array(self.model.predict(batch)['reconstruction'])      
            l=[]
        
            for i in range(0,batch.shape[0]):
                loss=AD_score_MSE(batch[i],sg_predict[i])
                l+=[loss]
    
            sg_loss= l
        
            self.signal_loss+=[sg_loss]

    return [self.br_loss,self.signal_loss]

In [7]:
Topo_2A_measure_1=Topo_2A_flat[500000:1012567,:]
Topo_t_measure_1=EB_weights[500000:1012567]
Topo_2A_measure_2=Topo_2A_flat[1012567+500000:,:]
Topo_t_measure_2=EB_weights[1012567+500000:]

e1=Model_Evaluator('/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_2A/Topo_2A_max_test_1/',Topo_2A_measure_1,Topo_t_measure_1,[Topo_2A_measure_1],[Topo_t_measure_1],111)
e1_loss=e1.calculate_loss('CKL')[0]

e2=Model_Evaluator('/eos/home-w/wsherman/AD_Work/ML_git_repo/AD_trigger_training/trained_models/L1_models/Topo_2A/Topo_2A_max_test_2/',Topo_2A_measure_2,Topo_t_measure_2,[Topo_2A_measure_2],[Topo_t_measure_2],111)
e2_loss=e2.calculate_loss('CKL')[0]

z1=np.zeros(500000)
z2=np.zeros(500000)


17104/17104 [==============================] - 26s 1ms/step


In [9]:

ad_scores=np.append(z1,e1_loss)
ad_scores1=np.append(ad_scores,z2)
ad_scores2=np.append(ad_scores1,e2_loss)
hf=h5py.File('For_Max4.h5','w')
hf.create_dataset('run_number',data=r_num)
hf.create_dataset('event_number',data=e_num)
hf.create_dataset('AD_Scores',data=ad_scores2)
hf.close()